## CS 5010: Semester Project


Pull last X trading days stock activity (X = 98?; based on visible values, not URL defined)

 - Convert closing stock data to % change (x) 
 - Common size performance (x)
 - Common size date range with google trends ( )

 - Send multiple ticker requests (x)
 - Need to limit the number of requests ( )
 - Look up ticker names ( )


In [ ]:
#############################################################################################
################################ CORE CODE ##################################################
#############################################################################################

In [167]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import csv 
import pandas as pd
import numpy as np

#############################################################################################
#Request ticker information
tickers = input("Please provide stock symbols separated by commas, max 5:") #aapl (Apple, inc.)
tickers = tickers.split(",")

#clean up white space and make all tickers upper case
for x in range(0,len(tickers)):
    tickers[x] = tickers[x].strip()
    tickers[x] = tickers[x].upper()


#############################################################################################
#establish global variables (unclear if this is necessary)
STOCKDATA = []
STOCKNAMES = []

for stock in tickers:
    try:
        stock_url = "https://finance.yahoo.com/quote/"+stock+"/history?p="+stock+""
        r = requests.get(stock_url)
        data = r.text
        soup = BeautifulSoup(data)

        contents = []

        for row in soup.find_all('tbody'):    
            for srow in row.find_all('tr'):
                for content in srow.find_all('td'):
                    if "Dividend" in content.text:
                        contents.pop() #removes prior entry of date of dividend data point and ignores dividend data point
                        continue
                    else:
                        contents.append(content.text)

#############################################################################################

#pulled all data into one long list, but there are 7 columns of data
#list comprehension to break out the full list in groupings by the 7 columns
        adj_contents = [contents[i * 7:(i + 1) * 7] for i in range((len(contents) + 7 - 1) // 7 )]

#the 7 columns of data, decided not to include most in this analysis
        date=[]
        #open_price=[]
        #high_price=[]
        #low_price=[]
        #close_price=[]
        adj_close_price=[]
        #volume = []
        percent_change = []
        normalized_returns = []

#assigning the values to the columns
        for i in range(0, len(adj_contents)): #change to a count for amount of days requested...up to 97
            date.append(adj_contents[i][0])
            #open_price.append(float(adj_contents[i][1]))
            #high_price.append(float(adj_contents[i][2]))
            #low_price.append(float(adj_contents[i][3]))
            #close_price.append(float(adj_contents[i][4]))
            adj_close_price.append(float(adj_contents[i][5].replace(",","")))
            #volume.append(float(adj_contents[i][6].replace(",","")))

#calculating the percent change from prior close to today        
        for j in range(0, len(adj_close_price)):
            if j == len(adj_close_price)-1:
                percent_change.append(0.00)
            else:
                percent_change.append(round(adj_close_price[j]/adj_close_price[j+1]-1,3))

#normalizing returns for comparison across companies
        normalizer = 100

        for x in range(len(percent_change), 0,-1):
            if x == len(percent_change):
                normalized_returns.append(normalizer)
            else:
                normalizer = round(normalizer * (1+percent_change[x-1]),3)
                normalized_returns.append(normalizer)

        normalized_returns.reverse()

#############################################################################################

#placing the values into a panda data frame
        x = pd.DataFrame({"Date": date, 
                          "Adj Close Price": adj_close_price,
                          "Percent Change": percent_change,
                          "Normalized Returns": normalized_returns})
        STOCKDATA.append(x)
        STOCKNAMES.append(stock)
        

    except Exception as e:
        print(e)
        #print("I'm sorry, {} was not a recognized stock ticker".format(stock))
    
#delay added to avoid triggering web scrapping blocks
    delay = [1, 2, 3, 4, 5]
    delay = np.random.choice(delay)

    sleep(delay)

#############################################################################################
#confirming what information was brought into the system
print("We recieved data for tickers:{}".format(STOCKNAMES))

We recieved data for tickers:['V', 'MSFT', 'AAPL', 'AMZN']


In [166]:
#just to represent that the information was brought down in ordered fashion/view results
print(STOCKNAMES[2])
print(STOCKDATA[2])

AMZN
            Date  Adj Close Price  Percent Change  Normalized Returns
0   Mar 20, 2020          1846.09          -0.019             103.910
1   Mar 19, 2020          1880.93           0.028             105.923
2   Mar 18, 2020          1830.00           0.012             103.038
3   Mar 17, 2020          1807.84           0.070             101.816
4   Mar 16, 2020          1689.15          -0.054              95.155
..           ...              ...             ...                 ...
95  Nov 01, 2019          1791.44           0.008             100.792
96  Oct 31, 2019          1776.66          -0.002              99.992
97  Oct 30, 2019          1779.99           0.010             100.192
98  Oct 29, 2019          1762.71          -0.008              99.200
99  Oct 28, 2019          1777.08           0.000             100.000

[100 rows x 4 columns]


-----
Scratch work

In [147]:
percent_change = []
normalized_returns = []


for j in range(0, len(adj_close_price)-1):
    if j == len(adj_close_price):
        percent_change.append(0.00)
    else:
        percent_change.append(round(adj_close_price[j]/adj_close_price[j+1]-1,3))

normalizer = 100

for x in range(len(percent_change)+1, 0,-1):
    if x == len(percent_change)+1:
        normalized_returns.append(normalizer)
    else:
        normalizer = round(normalizer * (1+percent_change[x-1]),3)
        normalized_returns.append(normalizer)

normalized_returns.reverse()
normalized_returns
#percent_change


[82.294,
 85.367,
 83.285,
 88.601,
 85.275,
 98.584,
 89.785,
 96.96,
 102.387,
 95.958,
 103.403,
 104.871,
 109.013,
 104.219,
 107.887,
 101.973,
 100.963,
 104.951,
 105.585,
 111.377,
 116.993,
 118.414,
 119.489,
 118.306,
 117.835,
 116.208,
 116.092,
 114.151,
 115.304,
 113.488,
 113.602,
 113.489,
 113.945,
 112.372,
 111.37,
 116.496,
 114.661,
 113.526,
 112.849,
 114.684,
 115.492,
 116.306,
 115.958,
 114.47,
 112.336,
 111.666,
 109.584,
 109.147,
 108.281,
 107.957,
 107.207,
 105.415,
 105.732,
 105.944,
 106.798,
 105.013,
 105.013,
 105.86,
 105.754,
 104.915,
 104.601,
 105.127,
 104.293,
 103.363,
 103.674,
 104.09,
 103.469,
 102.04,
 101.735,
 101.837,
 102.246,
 101.839,
 101.636,
 100.929,
 101.64,
 101.538,
 103.084,
 102.981,
 101.961,
 101.052,
 100.25,
 100.451,
 101.466,
 102.078,
 100.372,
 100.472,
 100.472,
 100.072,
 100.273,
 100.173,
 99.873,
 99.574,
 98.686,
 98.489,
 99.887,
 100.998,
 99.8,
 100]

############### Original ############### 


In [55]:
import requests
from bs4 import BeautifulSoup
import csv 
import pandas as pd

#############################################################################################

ticker = input("Please provide a stock symbol:") #aapl (Apple, inc.)
ticker = ticker.upper()

#############################################################################################

try:
    stock_url = "https://finance.yahoo.com/quote/"+ticker+"/history?p="+ticker+""
    r = requests.get(stock_url)
    data = r.text
    soup = BeautifulSoup(data)

    contents = []

    for row in soup.find_all('tbody'):
    
        for srow in row.find_all('tr'):
            for content in srow.find_all('td'):
                if "Dividend" in content.text:
                    contents.pop() #removes prior entry of date of dividend data point and ignores dividend data point
                    continue
                else:
                    contents.append(content.text)

#############################################################################################

#pulled all data into one long list, but there are 7 columns of data
#list comprehension to break out the full list in groupings by the 7 columns
    adj_contents = [contents[i * 7:(i + 1) * 7] for i in range((len(contents) + 7 - 1) // 7 )]

#the 7 columns of data
    date=[]
    open_price=[]
    high_price=[]
    low_price=[]
    close_price=[]
    adj_close_price=[]
    volume = []

#assigning the values to the columns
    for i in range(0, len(adj_contents)):
        date.append(adj_contents[i][0])
        open_price.append(adj_contents[i][1])
        high_price.append(adj_contents[i][2])
        low_price.append(adj_contents[i][3])
        close_price.append(adj_contents[i][4])
        adj_close_price.append(adj_contents[i][5])
        volume.append(adj_contents[i][6])

#############################################################################################

    #placing the values into a panda data frame
    x = pd.DataFrame({"Date": date, "Open Price": open_price, "High Price": high_price, "Low Price": low_price, "Close Price": close_price, "Adj Close Price": adj_close_price, "Volume": volume})
    print(x)

except:
    print("I'm sorry, that ticker was not recognized")

Date Open Price High Price Low Price Close Price Adj Close Price  \
0   Mar 20, 2020     154.82     158.26    145.67      146.83          146.83   
1   Mar 19, 2020     146.92     157.25    141.47      152.25          152.25   
2   Mar 18, 2020     148.46     150.68    139.80      148.48          148.48   
3   Mar 17, 2020     155.75     163.70    149.25      157.89          157.89   
4   Mar 16, 2020     151.14     165.31    147.98      152.01          152.01   
..           ...        ...        ...       ...         ...             ...   
93  Nov 05, 2019     179.07     179.56    175.18      176.37          175.82   
94  Nov 04, 2019     182.08     182.40    178.81      178.95          178.39   
95  Nov 01, 2019     180.13     181.32    179.56      180.93          180.37   
96  Oct 31, 2019     179.25     179.70    178.01      178.86          178.30   
97  Oct 30, 2019     177.79     179.42    176.39      179.25          178.69   

        Volume  
0   18,689,000  
1   19,362,800  


Show:
 - high 30 day
 - low 30 day
 - relative % over 30 days


Create Listing of S&P 500 Companies by Sector for optional selection?

Add suggested stock pairings?
